# Windows Scheduled Tasks Hunt

This huntbook is created with the Kestrel hunting blog: [Building a Huntbook to Discover Persistent Threats from Scheduled Windows Tasks](https://opencybersecurityalliance.org/posts/kestrel-2021-07-26/)

### 0. Overview

In Windows 10, scheduled tasks are managed by `svchost.exe`. In this cyber hunting practice, we will start from all scheduled tasks/services to hunt for persistent threats, which uses Windows schedule tasks to start during booting. We will rule out cyber risks of the monitored host.

The purpose of this hunt book is to give an example of STIX pattern matching and hunting from schedule tasks.

The data source of this hunt is a Windows machine with sysmon monitor running. The live trace is streamed to elasticsearch via logstash. And we install Kestrel (`kestrel-lang` and `kestrel-jupyter`) with STIX-Shifter elastic_ecs module (`stix-shifter-modules-elastic_ecs`) to connect to the real-time data source. More information on setting up the entire hunting stack can be found in our [blog](https://opencybersecurityalliance.org/posts/kestrel-2021-07-26/).

### 1. Listing All Schedule Tasks On The Monitored Machine

The monitored machine is running Windows 10, and scheduled tasks are run via `svchost.exe` with arguments `-k netsvcs -p -s Schedule`. We can start with a STIX pattern to match all processes with this command line. We randomly select a day to check.

In [1]:
#### THIS CELL IS FOR ERROR DEMONSTRATION PURPOSE, DELETE THIS CELL WHEN YOU REUSE THE HUNT ####

# create Kestrel variable scheduler with the list of scheduler processes
scheduler = GET process FROM stixshifter://host101
            WHERE [process:command_line = 'C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule']
            START t'2021-04-03T00:00:00Z' STOP t'2021-04-04T00:00:00Z'

[ERROR] DataSourceError: data source internal error: STIX-shifter translation failed. please test data source manually.

Unfortunately, current version (v3.4.7) of STIX Elastic ECS connector does not support matching process with their `command_line`. Let's fall back to a basic process attribute to get rough data from the data source. After `GET`, Kestrel will cache the retrieved data locally, and we can run more STIX patterns not limited by the incomplete STIX-Shifter implementation.

In [2]:
# first GET going through STIX shifter
svchost = GET process FROM stixshifter://host101
          WHERE [process:name = 'svchost.exe']
          START t'2021-04-03T00:00:00Z' STOP t'2021-04-04T00:00:00Z'
     
# second GET running locally against the returned/cached data from the first command
# no need to specify time range for GET from a Kestrel variable, check document of GET for more info
# https://kestrel.readthedocs.io/en/latest/language.html#get
scheduler = GET process
            FROM svchost
            WHERE [process:command_line = 'C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),process*,file*,directory*,ipv4-addr*,ipv6-addr*,mac-addr*,user-account*,network-traffic*
svchost,process,183,13579,15885,16068,16066,45124,24994,14762,14276,12925
scheduler,process,2,258,0,0,0,0,0,0,0,0


Great! There are 183 `svchost.exe` on the monitored machine on April 3rd, 2021. And two of them are task scheduler processes.

### 2. Finding Scheduled Tasks

The `svchost.exe -s Schedule` are system process to start scheduled tasks. It is easy to guess that concrete scheduled tasks are child processes of the two `svchost.exe` processes. Let's get them.

In [3]:
tasks = FIND process CREATED BY scheduler

DISP tasks ATTR name, command_line

name,command_line
AppHostRegistrationVerifier.exe,None
AppHostRegistrationVerifier.exe,C:\Windows\system32\AppHostRegistrationVerifier.exe
BackgroundDownload.exe,None
BackgroundDownload.exe,"""C:\Program Files (x86)\Microsoft Visual Studio\Installer\resources\app\ServiceHub\Services\Microsoft.VisualStudio.Setup.Service\BackgroundDownload.exe"""
CompatTelRunner.exe,C:\Windows\system32\compattelrunner.exe
CompatTelRunner.exe,None
CompatTelRunner.exe,C:\Windows\system32\compattelrunner.exe -maintenance
DeviceCensus.exe,C:\Windows\system32\devicecensus.exe
DeviceCensus.exe,None
DiskSnapshot.exe,None


### 3. Drilling Down on a Scheduled Task

While most process names are common, an uncommon one is `amcet.exe`. Neither Google or VirusTotal has information about this proprietary process. From its command line, we can guess this is a legitimate process owned by IBM. Let's pull out its details to check.

In [4]:
# get only the amcet processes from all scheduled tasks
amcet = GET process FROM tasks WHERE [process:name = 'amcet.exe']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),process*,file*,directory*,ipv4-addr*,ipv6-addr*,mac-addr*,user-account*,network-traffic*
amcet,process,24,165,0,0,0,0,0,0,0,0


The execution summary does not show any related records cached since they are already retrieved in the last hunt step when generating variable `tasks`. We can brute force check whether there are child processes and network activities of the processes in `amcet`.

In [5]:
# find and display their child processes
amcet_child = FIND process CREATED BY amcet
DISP amcet_child ATTR name, command_line

# find and display their network traffic
nt = FIND network-traffic CREATED BY amcet
DISP nt ATTR dst_ref.value, dst_port

name 
 command_line 
 
 
 
 
 conhost.exe 
 \??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1 
 
 
 conhost.exe 
 None 
 
 
 powershell.exe 
 powershell.exe -NoExit -Command - 
 
 
 powershell.exe 
 None 
 
 
 
 
 
 
 dst_ref.value 
 dst_port 
 
 
 
 
 9.148.5.93 
 443 
 
 
 
 Block Executed in 5 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 process* 
 file* 
 directory* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 user-account* 
 network-traffic* 
 
 
 
 
 amcet_child 
 process 
 45 
 321 
 2443 
 2488 
 2474 
 4104 
 1695 
 1689 
 1156 
 366 
 
 
 nt 
 network-traffic 
 23 
 23 
 1673 
 1673 
 1660 
 2386 
 1096 
 1094 
 671 
 77 
 
 
 *Number of related records cached.

The only IP that is accessed by `amcet` is an IBM internal IP, which looks safe. But there is still a chance that the IP is infected and the threat goes from it to the monitored Windows machine. The two child processes of `amcet` are powerful system management processes, and `conhost` with arguments `0xffffffff -ForceV1` can access kernel space and is used in many attacks. Let's further check whether there is any process spawned from the two.

In [6]:
# find child processes of amcet_child
amcet_cc = FIND process CREATED BY amcet_child
DISP amcet_cc ATTR name, command_line

# find files read/written/executed by amcet_child
# Kestrel v1.0.8 only resolves the generic relation LINKED into STIX 2.0 references,
# which is limited to execution relation between processes and executables
amcet_f = FIND file LINKED amcet_child
DISP amcet_f ATTR name, parent_directory_ref.path

name 
 parent_directory_ref.path 
 
 
 
 
 conhost.exe 
 C:\Windows\System32 
 
 
 powershell.exe 
 C:\Windows\System32\WindowsPowerShell\v1.0 
 
 
 
 Block Executed in 7 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 process* 
 file* 
 directory* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 user-account* 
 network-traffic* 
 
 
 
 
 amcet_cc 
 process 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 amcet_f 
 file 
 2 
 1563 
 4253 
 4251 
 4239 
 6982 
 2927 
 2921 
 1896 
 573 
 
 
 *Number of related records cached.

Fortunately, there is no spawned processes. And the only files associated with `amcet_child` are the executables of the processes.

We can further check other scheduled tasks such as `wscript.exe` and `setdata.exe`. We can also construct common STIX patterns for situations other than matching scheduled tasks. The STIX patterns can be reused in different hunts, as well as the part/overall of this huntflow.